In [2]:
import tensorflow as tf
import csv
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import imghdr
import IPython.display as display
import numpy as np
from pathlib import Path

In [3]:
def _int64_feature(value):
#Wrapper for inserting int64 features into Example proto.
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _bytes_feature(value):
#Wrapper for inserting bytes features into Example proto.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def image_example(image_string, label):
    image_shape = tf.image.decode_image(image_string).shape
    feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }
    return tf.train.Example(features=tf.train.Features(feature=feature))


record_file = 'test.tfrecords'
numbers = 10
buffer_size = 0
with tf.io.TFRecordWriter(record_file) as writer:
    for i in range(numbers):
        folder = i
        data_folder = Path("mnist data/trainingSample/trainingSample/%d" %folder)
        directory = os.fsencode(data_folder)
        for file in os.listdir(directory):
            filename = data_folder / os.fsdecode(file)
            image_string = open(filename,'rb').read()
            label = folder
            tf_example = image_example(image_string, label)
            writer.write(tf_example.SerializeToString())
            buffer_size +=1
#for line in str(image_example(image_string, label)).split('\n')[:15]:
#    print(line)
#print('...')


In [4]:
raw_image_dataset = tf.data.TFRecordDataset('test.tfrecords')

def _parse_image_function(example_proto):
    # Parse the input tf.Example proto using the dictionary above.
    # Create a dictionary describing the features.
    image_feature_description = {
        'height': tf.io.FixedLenFeature([], tf.int64),
        'width': tf.io.FixedLenFeature([], tf.int64),
        'depth': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
    }
    image_features = tf.io.parse_single_example(example_proto,image_feature_description)
    image = tf.io.decode_raw(image_features['image_raw'],tf.uint8)
    image.set_shape([28*28])
    image = tf.cast(image, tf.float32)*(1. / 255)
    image = tf.reshape(image,[28,28,1])
    label = tf.cast(image_features['label'],tf.uint64)
    label = tf.squeeze(label)
    return image,label

#raw_image_dataset = raw_image_dataset.shuffle(1000)
parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
#parsed_image_dataset = parsed_image_dataset.batch(64)
#parsed_image_dataset = parsed_image_dataset.prefetch(1)

In [5]:
for image, label in parsed_image_dataset.take(1):
    print(image.shape, label.shape)

InvalidArgumentError: Input to reshape is a tensor with 602 values, but the requested shape has 784
	 [[{{node Reshape}}]]